In [1]:
import numpy as np
import pandas as pd
import tensorflow
from tensorflow import keras

In [2]:
df = pd.read_csv('Data Sets/diabetes.csv')
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [3]:
X = df.iloc[:, :-1]
y = df.iloc[:, -1]

In [4]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_scaled = sc.fit_transform(X)
X_scaled

array([[ 0.63994726,  0.84832379,  0.14964075, ...,  0.20401277,
         0.46849198,  1.4259954 ],
       [-0.84488505, -1.12339636, -0.16054575, ..., -0.68442195,
        -0.36506078, -0.19067191],
       [ 1.23388019,  1.94372388, -0.26394125, ..., -1.10325546,
         0.60439732, -0.10558415],
       ...,
       [ 0.3429808 ,  0.00330087,  0.14964075, ..., -0.73518964,
        -0.68519336, -0.27575966],
       [-0.84488505,  0.1597866 , -0.47073225, ..., -0.24020459,
        -0.37110101,  1.17073215],
       [-0.84488505, -0.8730192 ,  0.04624525, ..., -0.20212881,
        -0.47378505, -0.87137393]])

In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size = 0.2, random_state = 1)

In [6]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense

In [7]:
model = Sequential()

model.add(Dense(32, activation = 'relu', input_dim = 8))
model.add(Dense(1, activation = 'sigmoid'))

C:\Users\Anindya Dolui\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [8]:
model.compile(loss = 'binary_crossentropy', optimizer = 'Adam', metrics = ['accuracy'])

model.fit(X_train, y_train, batch_size = 32, epochs = 100, validation_data = (X_test, y_test))

Epoch 1/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 1s 12ms/step - accuracy: 0.6300 - loss: 0.6599 - val_accuracy: 0.7532 - val_loss: 0.6072
Epoch 2/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7170 - loss: 0.5978 - val_accuracy: 0.7662 - val_loss: 0.5688
Epoch 3/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7573 - loss: 0.5435 - val_accuracy: 0.7922 - val_loss: 0.5428
Epoch 4/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7551 - loss: 0.5421 - val_accuracy: 0.7727 - val_loss: 0.5278
Epoch 5/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7568 - loss: 0.5265 - val_accuracy: 0.7597 - val_loss: 0.5143
Epoch 6/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7748 - loss: 0.5115 - val_accuracy: 0.7532 - val_loss: 0.5048
Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7829 - loss: 0.4961 - val_accuracy: 0.7532 - val_loss: 0.4984
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7820 - loss: 0.4849 - val_accuracy: 0.7532 - 

3. Total no of hidden layers

In [9]:
import keras_tuner as kt

In [10]:
def build_model(hp):

    model = Sequential()

    # this is first hidden layer, then we'll automate if further hidden layers are required or not
    model.add(Dense(88, activation = 'relu', input_dim = 8))
    
    # it'll create total 10 models where first model will have only 1 hidden layer,
    # second model will have 2 layers....thus 10th model will have 10 hidden layers
    for i in range(hp.Int('num_layers', min_value=1, max_value=10)):
        model.add(Dense(88, activation='relu'))
        
    model.add(Dense(1, activation='sigmoid'))    # output layer -> same as before
    
    model.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy', metrics = ['accuracy'])
    
    return model

In [11]:
tuner = kt.RandomSearch(
    build_model,
    objective = 'val_accuracy',
    max_trials = 5,
    directory = 'Hyperparameter Tuning json',
    project_name = 'No of Hidden Layers'
)

Reloading Tuner from Hyperparameter Tuning json\No of Hidden Layers\tuner0.json


In [12]:
tuner.search(X_train, y_train, epochs=5, validation_data=(X_test, y_test))

In [13]:
tuner.get_best_hyperparameters()[0].values

{'num_layers': 2}

In [14]:
model = tuner.get_best_models(num_models = 1)[0]

C:\Users\Anindya Dolui\anaconda3\Lib\site-packages\keras\src\saving\saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [15]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 88)             │           792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 88)             │         7,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 88)             │         7,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            89 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 16,545 (64.63 KB)

 Trainable params: 16,545 (64.63 KB)

 Non-trainable params: 0 (0.00 B)

In [16]:
model.fit(X_train, y_train, batch_size = 32, epochs = 100, initial_epoch = 6, validation_data = (X_test, y_test))

Epoch 7/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 2s 18ms/step - accuracy: 0.7939 - loss: 0.4379 - val_accuracy: 0.8117 - val_loss: 0.4822
Epoch 8/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7713 - loss: 0.4364 - val_accuracy: 0.7792 - val_loss: 0.4950
Epoch 9/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7457 - loss: 0.4711 - val_accuracy: 0.7922 - val_loss: 0.4919
Epoch 10/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.7896 - loss: 0.4579 - val_accuracy: 0.7987 - val_loss: 0.4979
Epoch 11/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8037 - loss: 0.3945 - val_accuracy: 0.7987 - val_loss: 0.4677
Epoch 12/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.8125 - loss: 0.3962 - val_accuracy: 0.8052 - val_loss: 0.4763
Epoch 13/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8188 - loss: 0.3888 - val_accuracy: 0.8052 - val_loss: 0.4753
Epoch 14/100
20/20 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.7977 - loss: 0.4084 - val_accuracy: 0.76

previous accuracy was 78% where no of hidden layers was only 1
but after tuning, it results the accurate no of hidden layers should be 2 to increase the overall accuracy now it's 99%